In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import keras
from keras import layers, regularizers
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [3]:
BASE_DIR = '../input/'
TRAIN_DIR = '500x500_cropped_clahe_128/500x500_cropped_clahe_128/'
TARGET_SIZE = (299, 299)

In [4]:
labels_df = pd.read_csv(os.path.join(BASE_DIR, 'labels.csv'))

fifty_fifty_subset = pd.concat([
    labels_df[labels_df['class']=='symptoms'].sample(n=9000, random_state=0),
    labels_df[labels_df['class']=='nosymptoms'].sample(n=9000, random_state=0)
]).sample(frac=1.0, random_state=0) # shuffle

split = 70*len(fifty_fifty_subset)//100
train_df = fifty_fifty_subset.iloc[:split]
valid_df = fifty_fifty_subset.iloc[split:]

In [6]:
common_flow_kwargs = dict(
    directory=os.path.join(BASE_DIR, TRAIN_DIR),
    x_col='file_name',
    y_col='class',
    target_size=TARGET_SIZE,
    batch_size=20,
    class_mode='binary',
)

train_gen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=360,
    fill_mode='nearest',
    horizontal_flip=True,
    rescale=1/255,
).flow_from_dataframe(
    dataframe=train_df,
    **common_flow_kwargs,
)

valid_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=valid_df,
    **common_flow_kwargs,
)

Found 12600 images belonging to 2 classes.
Found 5400 images belonging to 2 classes.


In [7]:
# create the base pre-trained model
# keras.applications.mobilenet_v2.MobileNetV2(input_shape=None, alpha=1.0, depth_multiplier=1, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(64, activation='elu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(32, activation='elu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(16, activation='elu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=output)

87916544/87910968 [==============================] - 7s 0us/step


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [12]:
# train the model on the new data for a few epochs
model.fit_generator(
      train_gen,
      steps_per_epoch=train_gen.n//train_gen.batch_size,
      epochs=5,
      validation_data=valid_gen,
      validation_steps=valid_gen.n//valid_gen.batch_size,
)

Epoch 1/50
630/630 [==============================] - 444s 706ms/step - loss: 0.6799 - acc: 0.5875 - val_loss: 0.7467 - val_acc: 0.5117
Epoch 2/50
630/630 [==============================] - 419s 666ms/step - loss: 0.6528 - acc: 0.6140 - val_loss: 0.7522 - val_acc: 0.5039
Epoch 3/50
630/630 [==============================] - 420s 666ms/step - loss: 0.6500 - acc: 0.6151 - val_loss: 0.7638 - val_acc: 0.4983
Epoch 4/50
630/630 [==============================] - 419s 664ms/step - loss: 0.6430 - acc: 0.6261 - val_loss: 0.7859 - val_acc: 0.5069
Epoch 5/50
630/630 [==============================] - 419s 665ms/step - loss: 0.6425 - acc: 0.6271 - val_loss: 0.7389 - val_acc: 0.5043
Epoch 6/50
630/630 [==============================] - 421s 668ms/step - loss: 0.6425 - acc: 0.6287 - val_loss: 0.7264 - val_acc: 0.5161
Epoch 7/50
446/630 [====================>.........] - ETA: 1:43 - loss: 0.6336 - acc: 0.6400

KeyboardInterrupt: 

In [13]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
      train_gen,
      steps_per_epoch=train_gen.n//train_gen.batch_size,
      epochs=20,
      validation_data=valid_gen,
      validation_steps=valid_gen.n//valid_gen.batch_size,
)